In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

month_start = 9
year_start = 2017

month_end = 9
year_end = 2022



curr_month  = month_start
curr_year = year_start

finalDF = pd.DataFrame([])

while (True):
    
    print("Scanning month = {} year = {} ".format(curr_month, curr_year))
    url = 'https://www.umpd.umd.edu/stats/incident_logs.cfm?year={}&month={}'.format(curr_year, curr_month)
    dfs = pd.read_html(url)
    crimeTable = dfs[0]

    crimeTable
    crimeTable.drop(columns=['Unnamed: 5'], inplace=True)


    # crimeTable
    crimeTable['location_desc'] = None
    crimeTable['date_occurred'] = None
    crimeTable['time_occurred'] = None

    crimeTable['date_reported'] = None
    crimeTable['time_reported'] = None

    toDrop = []
    for index, row in crimeTable.iterrows():

        if index%2 == 0:
            location = crimeTable.loc[index+1]['OCCURRED DATE TIME LOCATION']
            
            row['location_desc'] = location

            occurDT = str(row['OCCURRED DATE TIME LOCATION']).split(" ")            
            row['date_occurred'] = occurDT[0]
            try:                
                row['time_occurred']= occurDT[1]
            except:
                row['time_occurred'] = "{NO TIME}"

            reportDT = str(row['REPORT DATE TIME']).split(" ")
            row['date_reported'] = reportDT[0]
            row['time_reported']= reportDT[1]
            toDrop.append(index+1)

    crimeTable.drop(toDrop, inplace=True)

    # print(crimeTable)

    crimeTable = crimeTable[["UMPD CASE NUMBER", "date_occurred", "time_occurred", "location_desc", "date_reported", "time_reported", "TYPE", "DISPOSITION"]]
    crimeTable.set_axis(['case_number', 'date_occurred', 'time_occurred', 'location_desc', 'date_reported', 'time_reported','type', 'disposition' ], axis=1, inplace=True)

    curr_month+=1

    if curr_month == 13:
        curr_month = 1
        curr_year+=1

    finalDF = finalDF.append(crimeTable)
    print('append done')
    if curr_month > month_end and curr_year == year_end:
        break
    
finalDF
finalDF.to_csv('UMPD_crime_export_{}_{}_to_{}_{}.csv'.format(month_start, year_start, month_end, year_end))